In [17]:
import numpy as np
import matplotlib.pyplot as plt

"""
Projects a 3D voxel list onto a 2D mask by collapsing along the x-axis.

Parameters:
- cell_voxels (dict): Mapping of cell ID to list of (x, y, z) voxel coordinates.
- grid_shape (tuple): Dimensions of the 3D grid.

Returns:
- mask (np.array): 2D binary mask representing the projected area.
- area_um2 (float): Total area in µm² assuming 1 voxel = 2 µm.
"""
def project_voxels_to_2D(cell_voxels, grid_shape, projection_axis='z', voxel_size_um=2.0):
    # Choose which axes to project onto
    axis_map = {'x': (1, 2), 'y': (0, 2), 'z': (0, 1)}
    if projection_axis not in axis_map:
        raise ValueError("Invalid projection axis. Choose from 'x', 'y', 'z'.")
    ax1, ax2 = axis_map[projection_axis]

    # Create empty 2D mask
    proj_shape = (grid_shape[ax1], grid_shape[ax2])
    mask = np.zeros(proj_shape, dtype=np.uint8)

    # Flatten all voxel coordinates into one list
    all_voxels = np.concatenate(list(cell_voxels.values()))
    for voxel in all_voxels:
        i, j = voxel[ax1], voxel[ax2]
        if 0 <= i < proj_shape[0] and 0 <= j < proj_shape[1]:
            mask[i, j] = 1

    # Calculate area in µm²
    area_pixels = np.sum(mask)
    pixel_area = voxel_size_um ** 2
    total_area_um2 = area_pixels * pixel_area

    return mask, total_area_um2


In [18]:
"""
Displays the projected 2D spheroid mask and prints the area in µm².

Parameters:
- mask (np.array): 2D binary mask.
- area_um2 (float): Area in µm².
- title (str): Optional title for the plot.
"""
def show_projected_area(mask, area_um2, title="Projected 2D Spheroid Area"):
    plt.figure(figsize=(6, 6))
    plt.imshow(mask.T, cmap='gray', origin='lower')
    plt.title(f"{title}\nArea = {area_um2:.0f} µm²")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.tight_layout()
    plt.show()


In [19]:
import h5py
import numpy as np

"""
Loads cell voxel and grid data from a JLD2 file.

Parameters:
- file_path (str): Path to the `.jld2` simulation output file.

Returns:
- cell_voxels (dict): Mapping of cell IDs to voxel coordinate lists.
- grid_shape (tuple): Shape of the CPM grid.
"""
def load_cell_voxels_from_jld2(file_path):
    with h5py.File(file_path, "r") as f:
        cell_voxels = {}
        max_coord = np.array([0, 0, 0])

        for cell_id in f["cell_voxels"]:
            raw_voxels = f["cell_voxels"][cell_id][()]
            
            # Convert structured array to regular array
            if raw_voxels.dtype.names:
                voxels = np.stack([raw_voxels[name] for name in raw_voxels.dtype.names], axis=-1)
            else:
                voxels = raw_voxels

            if voxels.size > 0:
                max_coord = np.maximum(max_coord, voxels.max(axis=0))
            cell_voxels[int(cell_id)] = voxels

        grid_shape = tuple(max_coord + 1)

    return cell_voxels, grid_shape


In [5]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os

"""
Visualizes a 3D nutrient or field (vax) slice at a given depth `z`.

Parameters:
- filepath (str): Path to the HDF5 file containing the field.
- field_name (str): Name of the field to visualize (e.g., "oxygen").
- z (int): Slice position along z-axis.
"""
def visualize_vax_field(filepath, field_name, mode="slice", axis=2, slice_index=None, cmap='viridis', verbose = False):
    """
    Visualize a 3D vax field from a .jld2 file using h5py.

    Parameters:
    - filepath (str): Path to the .jld2 file (e.g. 'sim_output/vaxes/vaxes_step_00010.jld2')
    - field_name (str): Name of the field to visualize (e.g. 'oxygen')
    - mode (str): Visualization mode - 'slice' or 'mip' (maximum intensity projection)
    - axis (int): Axis to slice or project along (0=x, 1=y, 2=z)
    - slice_index (int or None): For 'slice' mode, the index of the slice to show. If None, will use middle.
    - cmap (str): Colormap to use (default 'viridis')
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")

    with h5py.File(filepath, "r") as f:
        dataset_path = f"vaxes/{field_name}"
        if dataset_path not in f:
            raise KeyError(f"Field '{field_name}' not found in {filepath}. Available fields: {list(f['vaxes'].keys())}")
        field = np.array(f[dataset_path])

    if mode == "slice":
        if slice_index is None:
            slice_index = field.shape[axis] // 2

        if axis == 0:
            img = field[slice_index, :, :]
        elif axis == 1:
            img = field[:, slice_index, :]
        else:
            img = field[:, :, slice_index]

        title = f"{field_name} slice @ axis={axis}, index={slice_index}"
        if verbose:
            np.set_printoptions(threshold=np.inf, linewidth=200, precision=3, suppress=True)
            print(field[slice_index, :, :])

    elif mode == "mip":
        img = np.max(field, axis=axis)
        title = f"{field_name} max projection along axis {axis}"
    elif mode == "min":
        img = np.min(field, axis=axis)
        title = f"{field_name} min projection along axis {axis}"

    else:
        raise ValueError("mode must be 'slice' or 'mip'")

    plt.figure(figsize=(6, 5))
    plt.imshow(img.T, cmap=cmap, origin='lower')
    plt.colorbar(label=field_name)
    #plt.title(title)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import imageio.v2 as imageio  # You can use imageio_ffmpeg for MP4
from natsort import natsorted
import h5py
import numpy as np


def extract_vax_field_image(filepath, field_name, mode="slice", axis=2, slice_index=None, cmap='viridis'):
    """
    Extracts a 2D image from a 3D voxel field stored in a .jld2 HDF5 file.

    Parameters:
    - filepath (str): Path to the .jld2 file containing simulation output.
    - field_name (str): Name of the field (e.g., "glucose", "oxygen") to extract.
    - mode (str): One of "slice", "mip" (maximum intensity projection), or "min".
    - axis (int): Axis along which to slice or project (0=x, 1=y, 2=z).
    - slice_index (int or None): Index of the slice. If None, the center slice is used.
    - cmap (str): Colormap used for visualization (used only if plotting externally).

    Returns:
    - img (np.ndarray): 2D array representing the selected slice or projection.
    
    Raises:
    - KeyError: If the field is not found in the file.
    - ValueError: If an invalid mode is provided.
    """
    with h5py.File(filepath, "r") as f:
        dataset_path = f"vaxes/{field_name}"
        if dataset_path not in f:
            raise KeyError(f"Field '{field_name}' not found in {filepath}. Available fields: {list(f['vaxes'].keys())}")
        field = np.array(f[dataset_path])

    if mode == "slice":
        if slice_index is None:
            slice_index = field.shape[axis] // 2
        if axis == 0:
            img = field[slice_index, :, :]
        elif axis == 1:
            img = field[:, slice_index, :]
        else:
            img = field[:, :, slice_index]
    elif mode == "mip":
        img = np.max(field, axis=axis)
    elif mode == "min":
        img = np.min(field, axis=axis)
    else:
        raise ValueError("mode must be 'slice' or 'mip' or 'min'")

    return img

def make_vax_field_gif(folder, field_name, output_file="output.gif", mode="slice", axis=2, slice_index=None, cmap='viridis', frame_duration=1.0):
    """
    Creates an animated GIF or MP4 from 2D projections or slices of a voxel field
    across multiple simulation steps stored in a folder.

    Parameters:
    - folder (str): Path to the folder containing `.jld2` step files.
    - field_name (str): Name of the field (e.g., "glucose", "oxygen") to visualize.
    - output_file (str): Output filename (must end with `.gif` or `.mp4`).
    - mode (str): "slice", "mip" (maximum intensity projection), or "min".
    - axis (int): Axis along which to slice or project.
    - slice_index (int or None): Specific slice index (None = center slice).
    - cmap (str): Matplotlib colormap name (e.g., "viridis", "plasma").
    - frame_duration (float): Duration (in seconds) of each frame in the output video.

    Saves:
    - A GIF or MP4 file showing the evolution of the field over time.

    Raises:
    - ValueError: If no `.jld2` files are found, or if output format is unsupported.
    """
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import imageio
    from natsort import natsorted
    import os
    import numpy as np
    import io

    files = natsorted([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".jld2")])
    if not files:
        raise ValueError("No .jld2 files found in the folder.")

    images = []
    for fpath in files:
        try:
            img = extract_vax_field_image(fpath, field_name, mode=mode, axis=axis, slice_index=slice_index)
            norm = plt.Normalize(vmin=np.min(img), vmax=np.max(img))
            cmap_fn = cm.get_cmap(cmap)

            # Plot to a high-resolution figure
            fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
            cax = ax.imshow(img.T, cmap=cmap_fn, origin='lower')
            ax.set_title(f"{field_name} | {os.path.basename(fpath)}")
            fig.colorbar(cax, ax=ax)

            # Save figure to buffer
            buf = io.BytesIO()
            plt.tight_layout()
            plt.savefig(buf, format='png')
            buf.seek(0)
            images.append(imageio.imread(buf))
            plt.close(fig)
        except Exception as e:
            print(f"Skipping {fpath}: {e}")

    # Save as GIF
    if output_file.endswith(".gif"):
        imageio.mimsave(output_file, images, format='GIF', duration=frame_duration)
    # Save as MP4
    elif output_file.endswith(".mp4"):
        imageio.mimsave(output_file, images, format='FFMPEG', fps=1.0 / frame_duration)
    else:
        raise ValueError("Output file must end with .gif or .mp4")

    print(f"Saved video to {output_file}")
